In [5]:
"""
Random Forest
Global Feature Selection 
    - X = 'Depth (m)', 'σ,v (kPa)', 'σ',v (kPa)', 'SBT (-)', 'SBTn (-)',
    'Mod. SBTn (-)', 'Oberhollenzer_classes', 'iso_classes_encoded'.

    
    - Y = 'qc (MPa)','fs (kPa)'.

Link: https://github.com/DowellChan/ResNetRegression
"""
#General Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, CSVLogger
from sklearn.model_selection import train_test_split
import time

'''
PreProcessing
''' 
#Loading Database
df=pd.read_csv(r'/content/drive/MyDrive/Thesis/Article_04_Soil_Classification/Databases/mmc1.csv')
df

<ipython-input-5-6ccf5fef3926>:31: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r'/content/drive/MyDrive/Thesis/Article_04_Soil_Classification/Databases/mmc1.csv')


,ID,test_type,basin_valley,Depth (m),qc (MPa),fs (kPa),u2 (kPa),Vs (m/s),qt (MPa),Rf (%),...,U2 (-),SBT (-),SBTn (-),Mod. SBTn (-),n,Ic (-),Ic SBT (-),Ib (-),EN_ISO_14688_classes,Oberhollenzer_classes
0,0,CPT,Zell basin,0.02,1.15,1.0,NaN,NaN,1.77,0.06,...,NaN,0.0,0.0,0.0,-0.12,2.56,2.23,25.61,NaN,NaN
1,0,CPT,Zell basin,0.04,2.85,1.0,NaN,NaN,2.99,0.03,...,NaN,0.0,0.0,0.0,0.33,1.23,2.02,256.60,NaN,NaN
2,0,CPT,Zell basin,0.06,4.80,1.0,NaN,NaN,4.76,0.06,...,NaN,0.0,0.0,0.0,0.28,1.16,1.80,264.66,NaN,NaN
3,0,CPT,Zell basin,0.08,6.40,6.0,NaN,NaN,6.12,0.07,...,NaN,0.0,0.0,0.0,0.27,1.12,1.69,275.17,NaN,NaN
4,0,CPT,Zell basin,0.10,6.90,6.0,NaN,NaN,6.91,0.10,...,NaN,0.0,0.0,0.0,0.27,1.11,1.65,267.92,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516974,1338,CPTu,Enns valley - Radtstadt basin,9.82,49.59,0.0,-0.6,NaN,49.68,0.00,...,-0.63,0.0,0.0,0.0,1.00,0.00,0.00,630.32,"Gr,sa+",1.0
2516975,1338,CPTu,Enns valley - Radtstadt basin,9.83,50.51,0.0,-0.4,NaN,51.45,0.00,...,-0.63,0.0,0.0,0.0,1.00,0.00,0.00,651.86,"Gr,sa+",1.0
2516976,1338,CPTu,Enns valley - Radtstadt basin,9.84,54.25,0.0,0.1,NaN,53.22,0.00,...,-0.62,0.0,0.0,0.0,1.00,0.00,0.00,673.37,"Gr,sa+",1.0
2516977,1338,CPTu,Enns valley - Radtstadt basin,9.85,54.91,0.0,0.9,NaN,54.66,0.00,...,-0.63,0.0,0.0,0.0,1.00,0.00,0.00,690.71,"Gr,sa+",1.0


In [6]:
# generate a descriptive table of the dataset
table = df.describe(include='all')

# display the table
print(table)

                  ID test_type    basin_valley     Depth (m)      qc (MPa)  \
count   2.516979e+06   2516979         2516979  2.516979e+06  2.516162e+06   
unique           NaN         4              35           NaN           NaN   
top              NaN       CPT  Salzburg basin           NaN           NaN   
freq             NaN   1615909         1064259           NaN           NaN   
mean    6.982726e+02       NaN             NaN  1.239890e+01  5.562390e+00   
std     3.924927e+02       NaN             NaN  1.043221e+01  8.484159e+00   
min     0.000000e+00       NaN             NaN  0.000000e+00 -8.610000e+00   
25%     3.550000e+02       NaN             NaN  4.800000e+00  1.000000e+00   
50%     7.050000e+02       NaN             NaN  1.021000e+01  2.640000e+00   
75%     1.051000e+03       NaN             NaN  1.715000e+01  6.260000e+00   
max     1.338000e+03       NaN             NaN  1.030000e+02  1.229000e+02   

            fs (kPa)       u2 (kPa)     Vs (m/s)      qt (MPa) 

In [7]:
# create a pivot table to summarize the data
table = pd.pivot_table(df, index=['test_type', 'basin_valley'], values=['Depth (m)', 'qc (MPa)', 'fs (kPa)', 'Vs (m/s)', 'γ (kN/m³)'])

# display the table
print(table)

                                   Depth (m)    Vs (m/s)    fs (kPa)  \
test_type basin_valley                                                 
CPT       Abtenau basin             3.142709         NaN   72.447502   
          Attersee-Mondsee basin    7.722643         NaN  109.402544   
          Bad Reichenhall basin     5.394721         NaN   80.180659   
          Berchtesgaden basin       2.202740         NaN   71.511009   
          Eisenstadt-Sopron basin  12.725000         NaN  119.632783   
...                                      ...         ...         ...   
SCPTu     Attersee-Mondsee basin    9.775000  171.243243   49.709161   
          Flachgau                 11.319150  177.820000   42.415619   
          Salzburg basin           15.886485  255.550388   48.867332   
          Walgau                   10.627502  207.206897   31.588434   
          Zell basin               14.633950  171.165591   33.307066   

                                    qc (MPa)  γ (kN/m³)  
test_

In [8]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=c94e685082326dfcc367a2d0c668af58885399fb177665f970da6fa018e024c3
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [15]:
df.columns

Index(['ID', 'test_type', 'basin_valley', 'Depth (m)', 'qc (MPa)', 'fs (kPa)',
       'u2 (kPa)', 'Vs (m/s)', 'qt (MPa)', 'Rf (%)', 'γ (kN/m³)', 'σ,v (kPa)',
       'u0 (kPa)', 'σ',v (kPa)', 'Qt (-)', 'Qtn (-)', 'Fr (%)', 'Bq (-)',
       'U2 (-)', 'SBT (-)', 'SBTn (-)', 'Mod. SBTn (-)', 'n', 'Ic (-)',
       'Ic SBT (-)', 'Ib (-)', 'EN_ISO_14688_classes',
       'Oberhollenzer_classes'],
      dtype='object')

In [32]:
grouped_df = df.groupby('test_type')[
    [
        # 'qc (MPa)', 
    #  'fs (kPa)', 
    #  'σ,v (kPa)',
     'Depth (m)'
      ]]
desc_df = grouped_df.describe().reset_index()
desc_df.to_csv('qc.csv')

In [25]:
grouped_df = df.groupby('test_type')[['qc (MPa)', 'fs (kPa)', 'σ,v (kPa)']]
desc_df = grouped_df.describe()

# stack the column index to convert it to a row index
stacked_df = desc_df.stack()

# pivot the dataframe to move the statistics to a row below the column names
pivoted_df = stacked_df.unstack(level=-1)

# add a row with the group names
pivoted_df = pd.concat([pd.DataFrame({'test_type': pivoted_df.index.get_level_values(0)}).T, pivoted_df])

# rename the index to remove the 'count' suffix from the column names
pivoted_df = pivoted_df.rename_axis(index={'qc (MPa)_count': 'qc (MPa)',
                                            'fs (kPa)_count': 'fs (kPa)',
                                            'σ,v (kPa)_count': 'σ,v (kPa)'})

# # remove the 'count' suffix from the column names
# pivoted_df.columns = [col[:-6] for col in pivoted_df.columns]

print(pivoted_df)


             0     1     2      3  (qc (MPa), count)  (qc (MPa), mean)  \
test_type  CPT  CPTu  SCPT  SCPTu                NaN               NaN   
CPT        NaN   NaN   NaN    NaN          1615909.0          5.593042   
CPTu       NaN   NaN   NaN    NaN           667183.0          4.037170   
SCPT       NaN   NaN   NaN    NaN           102614.0          6.107169   
SCPTu      NaN   NaN   NaN    NaN           131273.0          3.060553   

           (qc (MPa), std)  (qc (MPa), min)  (qc (MPa), 25%)  (qc (MPa), 50%)  \
test_type              NaN              NaN              NaN              NaN   
CPT               8.997027             -6.0              1.0              3.0   
CPTu              6.937019              0.0              0.0              2.0   
SCPT             10.597936             -2.0              0.0              2.0   
SCPTu             6.160872             -8.0              0.0              1.0   

           ...  (fs (kPa), 75%)  (fs (kPa), max)  (σ,v (kPa), count)